In [2]:
!nvidia-smi

Tue Feb  1 13:53:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-md5cpibq
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-md5cpibq
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=46e1ec7814071fc2cc8f694cd192e26cd6c6060b9f78014fba053a1e6728e91e
  Stored in directory: /tmp/pip-ephem-wheel-cache-x1ui2gce/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%cd /usr/local/

/usr/local


In [6]:
%ls

bin/        cuda-11@    games/               LICENSE.txt  setup.cfg
cuda@       cuda-11.0/  _gcs_config_ops.so/  licensing/   share/
cuda-10.0/  cuda-11.1/  include/             man@         src/
cuda-10.1/  etc/        lib/                 sbin/        xgboost/


In [7]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [8]:
!rm -rf cuda

In [9]:
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [11]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
 
#define SRAND_VALUE 2137
#define BLOCK_SIZE 128

// żródło https://www.olcf.ornl.gov/tutorials/cuda-game-of-life/

__global__ void ghostRows(int dim, int *grid)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x + 1;
    if (id <= dim)
    {
        grid[(dim+2)*(dim+1)+id] = grid[(dim+2)+id];
        grid[id] = grid[(dim+2)*dim + id];
    }
}


__global__ void ghostCols(int dim, int *grid)
{
    int id = blockDim.x * blockIdx.x + threadIdx.x;
    if (id <= dim+1)
    {
        grid[id*(dim+2)+dim+1] = grid[id*(dim+2)+1];
        grid[id*(dim+2)] = grid[id*(dim+2) + dim];   
    }
}


__global__ void GOL(int dim, int *grid, int *newGrid)
{
    int iy = blockDim.y * blockIdx.y + threadIdx.y + 1;
    int ix = blockDim.x * blockIdx.x + threadIdx.x + 1;
    int id = iy * (dim+2) + ix;
    int numNeighbors;
 
    if (iy <= dim && ix <= dim) {
        numNeighbors = grid[id+(dim+2)] + grid[id-(dim+2)]
                     + grid[id+1] + grid[id-1]             
                     + grid[id+(dim+3)] + grid[id-(dim+3)]
                     + grid[id-(dim+1)] + grid[id+(dim+1)];
        int cell = grid[id];
        // zasady gry w życia
        if (cell == 1 && numNeighbors < 2)
            newGrid[id] = 0;
        else if (cell == 1 && (numNeighbors == 2 || numNeighbors == 3))
            newGrid[id] = 1;
        else if (cell == 1 && numNeighbors > 3)
            newGrid[id] = 0;
        else if (cell == 0 && numNeighbors == 3)
            newGrid[id] = 1;
        else
            newGrid[id] = cell;
    }
}


int main()
{
    int i,j,iter;
    int* h_grid; //Grid on host
    int* d_grid; //Grid on device
    int* d_newGrid; //Second grid used on device only
    int* d_tmpGrid; //tmp grid pointer used to switch between grid and newGrid
 
    
    int dim = 1024;
    int steps = 1<<10;
 
    // tablica do gry z paskami granicznymi
    size_t bytes = sizeof(int)*(dim+2)*(dim+2);
    h_grid = (int*)malloc(bytes);
 

    /****************************
    M E T O D A  cudaMalloc
    ****************************/
 
    clock_t t;
    t = clock();

    cudaMalloc(&d_grid, bytes);
    cudaMalloc(&d_newGrid, bytes);
 
    // losowy stan początkowy
    srand(SRAND_VALUE);
    for(i = 1; i<=dim; i++) {
        for(j = 1; j<=dim; j++) {
            h_grid[i*(dim+2)+j] = rand() % 2;
        }
    }
 
    cudaMemcpy(d_grid, h_grid, bytes, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE,1);
    int linGrid = (int)ceil(dim/(float)BLOCK_SIZE);
    dim3 gridSize(linGrid,linGrid,1);
    dim3 cpyBlockSize(BLOCK_SIZE,1,1);
    dim3 cpyGridRowsGridSize((int)ceil(dim/(float)cpyBlockSize.x),1,1);
    dim3 cpyGridColsGridSize((int)ceil((dim+2)/(float)cpyBlockSize.x),1,1);
 
    // gra w życie
    for (iter = 0; iter<steps; iter++) {
        ghostRows<<<cpyGridRowsGridSize, cpyBlockSize>>>(dim, d_grid);
        ghostCols<<<cpyGridColsGridSize, cpyBlockSize>>>(dim, d_grid);
        GOL<<<gridSize, blockSize>>>(dim, d_grid, d_newGrid);
        // iterowanie
        d_tmpGrid = d_grid;
        d_grid = d_newGrid;
        d_newGrid = d_tmpGrid;
    }
 
    // wyniki
    cudaMemcpy(h_grid, d_grid, bytes, cudaMemcpyDeviceToHost);
    int total1 = 0;
    for (i = 1; i<=dim; i++) {
        for (j = 1; j<=dim; j++) {
            total1 += h_grid[i*(dim+2)+j];
        }
    }
    printf("Total Alive: %d\n", total1);

    memset(h_grid, 0, bytes);
    free(h_grid);

    t = clock() - t;
    double time_taken = ((double)t)/CLOCKS_PER_SEC;
    printf("Method cudaMalloc took %f seconds to execute \n", time_taken);
 
    /****************************
     M E T O D A   cudaHostAlloc
    ****************************/
 
    t = clock();
 
    cudaHostAlloc(&h_grid, bytes, cudaHostAllocMapped);
    cudaHostGetDevicePointer(&d_grid, h_grid, 0);
    cudaHostGetDevicePointer(&d_newGrid, h_grid, 0);
    cudaHostGetDevicePointer(&d_tmpGrid, h_grid, 0);

    // losowy stan początkowy
    srand(SRAND_VALUE);
    for(i = 1; i<=dim; i++) {
        for(j = 1; j<=dim; j++) {
            h_grid[i*(dim+2)+j] = rand() % 2;
        }
    }

    // gra w życie
    for (iter = 0; iter<steps; iter++) {
        ghostRows<<<cpyGridRowsGridSize, cpyBlockSize>>>(dim, d_grid);
        ghostCols<<<cpyGridColsGridSize, cpyBlockSize>>>(dim, d_grid);
        GOL<<<gridSize, blockSize>>>(dim, d_grid, d_newGrid);
        // iterowanie
        d_tmpGrid = d_grid;
        d_grid = d_newGrid;
        d_newGrid = d_tmpGrid;
    }
    cudaDeviceSynchronize();
 
    // wyniki
    cudaMemcpy(h_grid, d_grid, bytes, cudaMemcpyDeviceToHost);
    int total3 = 0;
    for (i = 1; i<=dim; i++) {
        for (j = 1; j<=dim; j++) {
            total3 += h_grid[i*(dim+2)+j];
        }
    }
    printf("Total Alive: %d\n", total3);
 
    memset(h_grid, 0, bytes);
    cudaFreeHost(h_grid);
 
    t = clock() - t;
    time_taken = ((double)t)/CLOCKS_PER_SEC;
    printf("Method cudaHostAlloc took %f seconds to execute \n", time_taken);
 


    /****************************
       M E T O D A   cudaMallocHost
    ****************************/
      
    t = clock();
 
    cudaMallocHost(&h_grid, bytes);
 
    // losowy stan początkowy
    srand(SRAND_VALUE);
    for(i = 1; i<=dim; i++) {
        for(j = 1; j<=dim; j++) {
            h_grid[i*(dim+2)+j] = rand() % 2;
        }
    }
 
    cudaMemcpy(d_grid, h_grid, bytes, cudaMemcpyHostToDevice);

    // gra w życie
    for (iter = 0; iter<steps; iter++) {
        ghostRows<<<cpyGridRowsGridSize, cpyBlockSize>>>(dim, d_grid);
        ghostCols<<<cpyGridColsGridSize, cpyBlockSize>>>(dim, d_grid);
        GOL<<<gridSize, blockSize>>>(dim, d_grid, d_newGrid);
        // iterowanie
        d_tmpGrid = d_grid;
        d_grid = d_newGrid;
        d_newGrid = d_tmpGrid;
    }
 
    // wyniki
    cudaMemcpy(h_grid, d_grid, bytes, cudaMemcpyDeviceToHost);
    int total2 = 0;
    for (i = 1; i<=dim; i++) {
        for (j = 1; j<=dim; j++) {
            total2 += h_grid[i*(dim+2)+j];
        }
    }
    printf("Total Alive: %d\n", total2);

    memset(h_grid, 0, bytes);
    cudaFreeHost(h_grid);
 
    t = clock() - t;
    time_taken = ((double)t)/CLOCKS_PER_SEC;
    printf("Method cudaMallocHost took %f seconds to execute \n", time_taken);


    cudaFree(d_grid);
    cudaFree(d_newGrid);
    cudaFree(d_tmpGrid);
    return 0;
}

Total Alive: 523969
Method cudaMalloc took 0.180845 seconds to execute 
Total Alive: 523969
Method cudaHostAlloc took 0.090583 seconds to execute 
Total Alive: 523969
Method cudaMallocHost took 0.089778 seconds to execute 

